In [1]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

In [2]:
def FPMI(dic_ispds):

    ###############################
    ### get variables from GUI ###


    ### Optic ###
    laser_power = dic_ispds["laser_power"]#source laser power
    
    ### DoF ###
    dof = dic_ispds["dof"]#CARM PRCL BS


    sweep_range_start = dic_ispds["sweep_range_start"]
    sweep_range_end = dic_ispds["sweep_range_end"]
    lin_or_log = dic_ispds["lin_or_log"]
    istransfunc = dic_ispds["istransfunc"]
    
    ###############################
    
    kat = finesse.kat()


    ### tugiyaru ###



    # In[108]:


    ### INF components ###
    #tabun atode kawaranai
    input_finesse = """
    ### FPMI

    const f1 16.881M
    const f2 45.0159M

    # Input optics
    l i1 %s 0 n0
    s s_eo0 0 n0 n_eo1
    mod eom1 16.881M 0.3 1 pm n_eo1 n_eo2
    s s_eo1 0 n_eo2 n_eo3
    mod eom2 45.0159M 0.3 1 pm n_eo3 n_eo4
    s s_eo2 0 n_eo4 n1

    # Michelson
    bs bs1 0.5 0.5 0 45 n1 n2 n3 n4
    s lx 26.6649 n3 nx1
    s ly 23.3351 n2 ny1


    # X arm
    m mx1 0.996 0.004 0 nx1 nx2
    s sx1 3000 nx2 nx3
    m mx2 0.99995 5e-06 0 nx3 nxTRANS

    # Y arm
    m my1 0.996 0.004 90 ny1 ny2
    s sy1 3000 ny2 ny3
    m my2 0.99995 5e-06 90 ny3 nyTRANS

    """ % (laser_power)


    # In[109]:


    ### atode jissou

    # photodetector n4
    #pd0 photodetector n1

    #### PDs ####
    ## REFL

    #pd1 I1 $f1 0 n1
    #pd1 Q1 $f1 90 n1
    #pd1 I2 $f2 0 n1
    #pd1 Q2 $f2 90 n1


    ## AS
    #pd1 I1 $f1 0 n4
    #pd1 Q1 $f1 90 n4
    #pd1 I2 $f2 0 n4
    #pd1 Q2 $f2 90 n4

    ## REFL amplitude
    #ad CR0 0 n2*
    #ad SB1 16.881M n2*
    #ad SB2 45.0159M n2*

    ## AS amplitude
    #ad CR0 0 n4
    #ad SB1 16.881M n4
    #ad SB2 45.0159M n4


    # In[110]:
    if(istransfunc==True):
        
        ### PDH signal
        #pd1 REFL_I 40k 0 n_in
        #pd1 REFL_Q 40k 90 n_in
        #xaxis m1 phi lin -1 1 500
        #yaxis lin abs

        ###PDs location
        pd1_code = """
        # Heterodyne signals
        #pd1 I2 $f2 0 n1
        pd1 Q2 $f2 90 n4
        """
        input_finesse += pd1_code
        
        ### transfer
        trasfer_code = """
        ### Transfer function
        pd2 inphase 40k 0 10 n_in
        xaxis sig1 f %s .01 100 400
        put inphase f2 $x1
        fsig sig1 mx1 10 0
        yaxis lin db:deg
        """ % (lin_or_log)
        input_finesse += trasfer_code
        
    if(istransfunc==False):
        
        ### PDs location ###

        if(dic_ispds["REFL"]==True):
            pd_text = "n1"
            n1_pd = """
        # DC PDs
        pd0 REFL n1
        ad CR0 0 n1
        ad SB1 16.881M n1
        ad SB2 45.0159M n1
            """
            input_finesse = input_finesse+n1_pd
        if(dic_ispds["AS"]==True):
            n4_pd = """
        #Amplitude detectors
        pd0 AS n4
        ad CR0 0 n4
        ad SB1 16.881M n4
        ad SB2 45.0159M n4
            """
            input_finesse = input_finesse+n4_pd
        
        ### DoF ###

        if(dof=="DARM"):
            darm_sweep = """
        # DARM scan
        xaxis mx2 phi %s %s %s 500
        put* my2 phi $mx1
        #diff mx2 phi
        yaxis abs

            """ % (lin_or_log, dic_ispds["sweep_range_start"], dic_ispds['sweep_range_end'])
            input_finesse = input_finesse + darm_sweep

        elif(dof=="CARM"):
            carm_sweep = """
        # CARM scan
        xaxis* mx2 phi %s %s %s 500
        put* my2 phi $x1
        #diff mx2 phi
        yaxis abs
            """ % (lin_or_log,dic_ispds['sweep_range_start'],dic_ispds['sweep_range_end'])
            input_finesse = input_finesse + carm_sweep

        elif(dof=="BS"):
            bs_sweep = """
        # BS scan
        xaxis* mx1 phi %s %s %s 500
        put* my1 phi $mx1
        put* mx2 phi $x1
        put* my2 phi $mx1
        #diff mx1 phi
        yaxis abs
            """ % (lin_or_log,dic_ispds['sweep_range_start'],dic_ispds['sweep_range_end'])
            input_finesse = input_finesse + bs_sweep
        else:
            pass

    code = input_finesse
    print(code)
    kat.parse(code)

    out = kat.run()
    fig = out.plot(detectors=["REFL"])
    fig2 = out.plot(detectors=["AS"])
    #out.plot()


In [3]:
import PySimpleGUI as sg    
from pykat import finesse
from pykat.commands import * 

###########################################
# def finesse script?
# def MI():



###########################################


SYMBOL_UP =    '▲'
SYMBOL_DOWN =  '▼'

## collapce section2
def collapse(layout, key):
    """
    Helper function that creates a Column that can be later made hidden, thus appearing "collapsed"
    :param layout: The layout for the section
    :param key: Key used to make this seciton visible / invisible
    :return: A pinned column that can be placed directly into your layout
    :rtype: sg.pin
    """
    return sg.pin(sg.Column(layout, key=key))

section2 = [
            [sg.Text('laser_power [W]'), sg.Input(key='laser_power', default_text='1')],
            ### slider 0.1 toka hyoujidekinai
            [sg.Checkbox('istransfunc', size=(10, 1), key='istransfunc')],
            [sg.Text('xaxis'),sg.Combo(('lin', 'log'),default_value='lin', size=(20, 1),key='lin_or_log')], 
            [sg.Text('sweep range'),sg.Input(key='sweep_range_start', default_text='-180', enable_events=True),
            sg.Text("to"),sg.Input(key='sweep_range_end', default_text='180', enable_events=True)],
           ]

sg.theme('Default1')

tab1_layout =  [
                ]

tab2_layout = [
                [sg.Image("./Fabry_Perot_MI_eom_drawing.png", key="imageContainer", size=(800,600))],
                [sg.Text('DoF'),sg.Combo(('DARM', 'CARM', 'BS'),default_value='DARM', size=(20, 1),key='dof')],
                ## PDs checkbox
                [sg.Text('check PDs')],
                [sg.Checkbox('REFL', size=(5, 1), key='REFL'),sg.Checkbox('AS', size=(5, 1), key='AS'),
                sg.Checkbox('nytrans', size=(5, 1), key='nytrans'),sg.Checkbox('nxtrans', size=(5, 1), key='nxtrans'),
                sg.Checkbox('n2', size=(5, 1), key='n2'),sg.Checkbox('n3', size=(5, 1), key='n3'),
                sg.Checkbox('ny1', size=(5, 1), key='ny1'),sg.Checkbox('nx1', size=(5, 1), key='nx1'),
                sg.Checkbox('ny2', size=(5, 1), key='ny2'),sg.Checkbox('nx2', size=(5, 1), key='nx2'),
                sg.Checkbox('ny3', size=(5, 1), key='ny3'),sg.Checkbox('nx3', size=(5, 1), key='nx3'),
                ],
                [sg.Button('Plot', button_color=('white', 'black'), key='FPMI')],
                #### Section 2 part ####             
                [sg.T(SYMBOL_DOWN, enable_events=True, k='-OPEN SEC2-', text_color='purple'),
                 sg.T('Section 2', enable_events=True, text_color='purple', k='-OPEN SEC2-TEXT')],  
                [collapse(section2, '-SEC2-')]
                ]

layout = [[sg.TabGroup([[sg.Tab('Tab 1', tab1_layout), sg.Tab('FPMI_TAB', tab2_layout)]])],    
          [sg.Checkbox('Hide Section 2', enable_events=True, key='-OPEN SEC2-CHECKBOX')],
          [sg.Button('Exit')] 
         ]    

window = sg.Window('My window with tabs', layout, default_element_size=(12,1))    

opened2 = True

while True:    
    event, values = window.read()

    print(event,values)
    if window == sg.WIN_CLOSED:     # if all windows were closed
        break
    if event == "Exit":           # always,  always give a way out!    
        break
    if event.startswith('-OPEN SEC2-'): ## show/hide section2
        opened2 = not opened2
        window['-OPEN SEC2-'].update(SYMBOL_DOWN if opened2 else SYMBOL_UP)
        window['-OPEN SEC2-CHECKBOX'].update(not opened2)
        window['-SEC2-'].update(visible=opened2)
    if event == 'sweep_range_start' and values['sweep_range_start'] and values['sweep_range_start'][-1] not in ('0123456789.-'):
        window['sweep_range_start'].update(values['sweep_range_start'][:-1])
    if event == 'sweep_range_end' and values['sweep_range_end'] and values['sweep_range_end'][-1] not in ('0123456789.-'):
        window['sweep_range_end'].update(values['sweep_range_end'][:-1])
    if event == "FPMI":           # FPMI  
        ###############################
        ### get variables from GUI ###
        ###############################

        ### Optic ###
        ##laser_power = values['laser_power']#source laser power
    
        ### what INF ###
        ##what_inf = values['what_inf']

        ### PD location ###
        dic_ispds = {
            ### DoF
            "dof":values['dof'],
            ### PDs
            "REFL":values['REFL'],
            "AS":values['AS'],
            "nytrans":values['nytrans'],
            "nxtrans":values['nxtrans'],
            "n2":values['n2'],
            "n3":values['n3'],
            "ny1":values['ny1'],
            "nx1":values['nx1'],
            "ny2":values['ny2'],
            "nx2":values['nx2'],
            "ny3":values['ny3'],
            "nx3":values['nx3'],
            ### advanced setting
            "laser_power":values['laser_power'],
            "istransfunc":values['istransfunc'],
            "lin_or_log":values['lin_or_log'],
            "sweep_range_start":values['sweep_range_start'],
            "sweep_range_end":values['sweep_range_end'],
        }

        ########
        #start_deg = values['start_deg']
        #end_deg = values['end_deg']  
        ########
        
        
        FPMI(dic_ispds)
    
window.close()         

                                              ..-
    PyKat 1.2.1           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat

None {'dof': None, 'REFL': None, 'AS': None, 'nytrans': None, 'nxtrans': None, 'n2': None, 'n3': None, 'ny1': None, 'nx1': None, 'ny2': None, 'nx2': None, 'ny3': None, 'nx3': None, 'laser_power': None, 'istransfunc': None, 'lin_or_log': None, 'sweep_range_start': None, 'sweep_range_end': None, 0: None, 1: None, '-OPEN SEC2-CHECKBOX': None}


AttributeError: 'NoneType' object has no attribute 'startswith'

In [ ]:
# outを出力したいとき
# 作ったkatファイルを出力したいとき
import PySimpleGUI as sg

sg.theme('Dark Red')

layout = [[sg.Text('Browse to a file')],
          [sg.Input(key='-FILE-', visible=False, enable_events=True), sg.FileBrowse()]
         ]

event, values = sg.Window('File Compare', layout).read(close=True)

fname_output = sg.popup_get_file('What is the name of output file?', save_as=True, file_types=(("ALL Files", "*.txt"),))
fname_kat = sg.popup_get_file('What is the name of kat file?', save_as=True, file_types=(("ALL Files", "*.txt"),))

print(f'You chose: {values["-FILE-"]}')
print(f'You chose: {fname_output}')
print(f'You chose: {fname_kat}')

In [1]:
# 作ったkatファイルを出力したいとき
# out は、フィネスを走らせていないので書き出していない。けど大体同じ感じOK
import PySimpleGUI as sg
import sys

sg.theme('Dark Red')

kat = """
const f1 16.881M
    const f2 45.0159M

    # Input optics
    l i1 %s 0 n0
    s s_eo0 0 n0 n_eo1
    mod eom1 16.881M 0.3 1 pm n_eo1 n_eo2
    s s_eo1 0 n_eo2 n_eo3
    mod eom2 45.0159M 0.3 1 pm n_eo3 n_eo4
    s s_eo2 0 n_eo4 n1

    # Michelson
    bs bs1 0.5 0.5 0 45 n1 n2 n3 n4
    s lx 26.6649 n3 nx1
    s ly 23.3351 n2 ny1


    # X arm
    m mx1 0.996 0.004 0 nx1 nx2
    s sx1 3000 nx2 nx3
    m mx2 0.99995 5e-06 0 nx3 nxTRANS

    # Y arm
    m my1 0.996 0.004 90 ny1 ny2
    s sy1 3000 ny2 ny3
    m my2 0.99995 5e-06 90 ny3 nyTRANS
"""


layout = [
    [sg.Text('Which data you output?')],
    [sg.Checkbox('kat file', size=(5, 1), key='output_kat'), sg.Checkbox('plotdata', size=(5, 1), key='output_plotdata')],
    [sg.Button('Plot', button_color=('white', 'black'), key='event_FPMI')]
]

window = sg.Window('test dayo', layout)
fname_output=""
fname_kat=""

while True:    
    event, values = window.read()

    print(event,values)
    if event == sg.WIN_CLOSED:     # if all windows were closed
        break
        
    if event == "event_FPMI":           # FPMI  

        if values['output_kat'] == True:
            fname_kat = sg.popup_get_file('Select the output name for kat file?', save_as=True, file_types=(("ALL Files", "*.txt"),))

            try:
                f = open(fname_kat, 'x')
                f.writelines(kat)
                f.close()
            except FileExistsError:
                sg.popup_ok("Error : there is a file %s" % fname_kat)
            except Exception:
                sg.popup_ok("Unexpected error:", sys.exc_info()[0])
            
        if values['output_plotdata'] == True:
            fname_plotdata = sg.popup_get_file('Select the output name for plot data?', save_as=True, file_types=(("ALL Files", "*.txt"),))
        break
        
window.close()         

print(f'You chose: {fname_output}')
print(f'You chose: {fname_kat}')

event_FPMI {'output_kat': True, 'output_plotdata': False}
You chose: 
You chose: /users/MIF/gw-finesse/aaaa.txt


In [6]:
import pykat

pykat.init_pykat_plotting()

from pykat import finesse

code = """
l l1 1 0 0 n1 
s s1 10 1 n1 n2
m m1 0.9 0.1 0 n2 n3
s s2 10 1 n3 n4
m m2 0.91 0.09 0 n4 n5

ad refl 0 n2
ad circ 0 n4
ad tran 0 n5
pd pd_cav n3

noplot refl

xaxis m2 phi lin 0 360 720
yaxis abs:deg
"""

kat = finesse.kat()
kat.parse(code)

out = kat.run()

--------------------------------------------------------------
Running kat - Started at 2020-08-21 15:52:55.499150
Used Finesse 2.3.1 at /home/controls/miniconda2/envs/mifsim/bin/kat

Finished in 0.0675364 seconds


100% | ETA:  0:00:00 | Calculating                                             

In [25]:
x=[101, 102, 103, 104, 105]
y=[1,2,3,4,5]
z=
a=
b=

plotdata=[["101 1"], ["102 2"], ["103 3"], ["104 4"], ["105 5"]]

try:
        f = open("hoge.txt", 'w')
        #f.writelines(plotdata)

        for key in dic_fpmi_ispds.keys():
            if dic_fpmi_ispds.keys[key] == True:
                kokode de-ta tukuru
                
        for i in plotdata:
            f.writelines(i)
            f.write('\n')
        
        f.close()
except FileExistsError:
        sg.popup_ok("Error : there is a file %s" % fname_kat)


In [ ]:
x=[101, 102, 103, 104, 105]
y=[1,2,3,4,5]

plotdata=["101 1", "102 2"]

In [142]:
import numpy as np

x=[101, 102, 103, 104, 105]
y=[1,2,3,4,5]
z=[1,2,3,4,5]
#plotdata=["101 1", "102 2", "103 3", '104 4', "105 5"]

x2=[x]
x2.append(y)
x2.append(z)
print(x2)

L = np.asarray(x2).T
x=L.tolist()
print(x)

#a=[[101, 1, 1], [102, 2, 2], [103, 3, 3], [104, 4, 4], [105, 5, 5]]
plotdata=[" ".join(map(str, i)) for i in x]
print(plotdata) 

[[101, 102, 103, 104, 105], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]
[[101, 1, 1], [102, 2, 2], [103, 3, 3], [104, 4, 4], [105, 5, 5]]
['101 1 1', '102 2 2', '103 3 3', '104 4 4', '105 5 5']


In [140]:
f = open("hoge.txt", 'w')

#for key in dic_fpmi_ispds.keys():
#    if dic_fpmi_ispds.keys[key] == True:
#        kokode de-ta tukuru

for i in plotdata:
    f.writelines(i)
    f.write('\n')
f.close()

In [135]:
a=[21]
a.append(5)
a.append(555)
a

[21, 5, 555]

In [119]:
a=[1, 2, 3]
#' '.join(str(a))

["%s" % i for i in a]

['1', '2', '3']

In [120]:
a=[[101, 1, 1], [102, 2, 2], [103, 3, 3], [104, 4, 4], [105, 5, 5]]


["%s" % i for i in a]

['[101, 1, 1]', '[102, 2, 2]', '[103, 3, 3]', '[104, 4, 4]', '[105, 5, 5]']

In [129]:
a=[1, 2, 3]
print(str(a))

b=' '.join(str(a))
b

[1, 2, 3]


'[ 1 ,   2 ,   3 ]'

In [125]:
" ".join(str(i) for i in [0,90,1234,5678])

'0 90 1234 5678'

In [108]:
L = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
#L2 = list(zip(*L))
#L2

# How to do transpose
# https://hacknote.jp/archives/56663/

L3 = [list(x) for x in zip(*L)]
L3

[[1, 4, 7], [2, 5, 8], [3, 6, 9]]

In [107]:
a=map(str, L3)
print('-'.join(a))

[1, 4, 7]-[2, 5, 8]-[3, 6, 9]


In [82]:
L = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
for i in zip(L):
    print(i)

for i in zip(*L):
    print([i])

([1, 2, 3],)
([4, 5, 6],)
([7, 8, 9],)
[(1, 4, 7)]
[(2, 5, 8)]
[(3, 6, 9)]
